In [1]:
#importing the libraries

import sys, os
import pandas as pd
import numpy as np

from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Conv2D, MaxPooling2D, BatchNormalization,AveragePooling2D
from keras.losses import categorical_crossentropy
from keras.optimizers import Adam
from keras.regularizers import l2
from keras.utils import np_utils

In [5]:
#Loading the dataset

df=pd.read_csv('fer2013.csv')

In [6]:
#Dividing data into training and test set

num_classes = 7
img_rows,img_cols = 48,48
batch_size = 32

X_train,train_y,X_test,test_y=[],[],[],[]

for index, row in df.iterrows():
    val=row['pixels'].split(" ")
    try:
        if 'Training' in row['Usage']:
           X_train.append(np.array(val,'float32'))
           train_y.append(row['emotion'])
        elif 'PublicTest' in row['Usage']:
           X_test.append(np.array(val,'float32'))
           test_y.append(row['emotion'])
    except:
        print(f"error occured at index :{index} and row:{row}")


num_features = 64
num_labels = 7
batch_size = 64
epochs = 25
width, height = 48, 48


X_train = np.array(X_train,'float32')
train_y = np.array(train_y,'float32')
X_test = np.array(X_test,'float32')
test_y = np.array(test_y,'float32')

In [7]:
#Categorically encoding the columns 

train_y=np_utils.to_categorical(train_y, num_classes=num_labels)
test_y=np_utils.to_categorical(test_y, num_classes=num_labels)

In [9]:
#Normalizing data between 0 and 1

X_train -= np.mean(X_train, axis=0)
X_train /= np.std(X_train, axis=0)

X_test -= np.mean(X_test, axis=0)
X_test /= np.std(X_test, axis=0)

X_train = X_train.reshape(X_train.shape[0], 48, 48, 1)

X_test = X_test.reshape(X_test.shape[0], 48, 48, 1)

In [10]:
#Designing the CNN

#1st convolution layer

model = Sequential()

# Block-1

model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(32,(3,3),padding='same',kernel_initializer='he_normal',input_shape=(img_rows,img_cols,1)))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-2 

model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(64,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-3

model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(128,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-4 

model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Conv2D(256,(3,3),padding='same',kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2)))
model.add(Dropout(0.2))

# Block-5

model.add(Flatten())
model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-6

model.add(Dense(64,kernel_initializer='he_normal'))
model.add(Activation('elu'))
model.add(BatchNormalization())
model.add(Dropout(0.5))

# Block-7

model.add(Dense(num_classes,kernel_initializer='he_normal'))
model.add(Activation('softmax'))

print(model.summary())

Model: "sequential_2"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_16 (Conv2D)           (None, 48, 48, 32)        320       
_________________________________________________________________
activation_22 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_20 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
conv2d_17 (Conv2D)           (None, 48, 48, 32)        9248      
_________________________________________________________________
activation_23 (Activation)   (None, 48, 48, 32)        0         
_________________________________________________________________
batch_normalization_21 (Batc (None, 48, 48, 32)        128       
_________________________________________________________________
max_pooling2d_8 (MaxPooling2 (None, 24, 24, 32)       

In [11]:
#Compliling the model

model.compile(loss='categorical_crossentropy',
              optimizer = Adam(lr=0.001),
              metrics=['accuracy'])

In [12]:
#Training the model

model.fit(X_train, train_y,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(X_test, test_y),
          shuffle=True)

Epoch 1/25
449/449 [==============================] - 44s 22ms/step - loss: 2.4288 - accuracy: 0.2214 - val_loss: 1.5657 - val_accuracy: 0.4218
Epoch 2/25
449/449 [==============================] - 9s 21ms/step - loss: 1.5767 - accuracy: 0.3903 - val_loss: 1.3365 - val_accuracy: 0.4904
Epoch 3/25
449/449 [==============================] - 9s 21ms/step - loss: 1.3765 - accuracy: 0.4727 - val_loss: 1.2614 - val_accuracy: 0.5286
Epoch 4/25
449/449 [==============================] - 9s 21ms/step - loss: 1.2865 - accuracy: 0.5140 - val_loss: 1.2027 - val_accuracy: 0.5411
Epoch 5/25
449/449 [==============================] - 9s 21ms/step - loss: 1.2202 - accuracy: 0.5465 - val_loss: 1.1700 - val_accuracy: 0.5564
Epoch 6/25
449/449 [==============================] - 10s 21ms/step - loss: 1.1641 - accuracy: 0.5668 - val_loss: 1.1269 - val_accuracy: 0.5743
Epoch 7/25
449/449 [==============================] - 10s 21ms/step - loss: 1.0994 - accuracy: 0.5946 - val_loss: 1.1285 - val_accuracy: 0.5

In [13]:
#Saving the  model to  use it later on
fer_json = model.to_json()
with open("fer2.json", "w") as json_file:
    json_file.write(fer_json)
model.save_weights("fer2.h5")